In [91]:
import os
import os.path
import pandas as pd
import pickle
from PIL import Image, ImageDraw
import re

In [92]:
def readPickle(filename):

    # reload a file to a variable
    with open(filename, 'rb') as file:
        a_dict1 =pickle.load(file)
        
    return a_dict1

In [93]:
def drawPicture(name, xfFix, yfFix, xfTra, yfTra, folder):
    
    name = nameJude(name, folder)
    image = Image.open(r'..//img/back.png')
    
    draw = ImageDraw.Draw(image)

    drawObject(draw, xfFix, yfFix, 0, 0, 255, 255, 30, 0, "Fix")
    drawObject(draw, xfTra, yfTra, 255, 0, 0, 255, 255, 0, "Tracing")
        
    image.save(r'..//' + folder + '/' + name + '.png')

In [94]:
def drawObject(draw, xf, yf, eR, eG, eB, lR, lG, lB, kind):
    
    bxf = 0 
    byf = 0
    
    q = int(255/len(xf))

    #画像に線を引く
    for i in range(len(xf)):  
        for j in range(1, len(xf[i])):
            if bxf is not 0: 
                draw.line((bxf, byf, xf[i][j], yf[i][j]), fill=(lR2, lG2, lB2), width=10)
                bxf = 0
                byf = 0
            
            draw.line((xf[i][j-1], yf[i][j-1], xf[i][j], yf[i][j]), fill=(lR, lG, lB), width=10)
            
            if j is len(xf[i])-1: 
                bxf=xf[i][j]
                byf=yf[i][j]
                lR2 = lR
                lG2 = lG
                lB2 = lB
            
        if len(xf[i]) is not 0: draw.ellipse((xf[i][0]-10, yf[i][0]-10, xf[i][0]+10, yf[i][0]+10), 
                                                                      fill=(eR, eG, eB), outline=None)
            
        lR = lR - q
        if kind is "Fix": lB = lB + q

In [95]:
#ファイル名を検索して、決定するメソッド
def nameJude(name, folder):
    
    #保存フォルダが持つファイルの取得
    files = os.listdir(r'..//' + folder)
    
    #保存できない記号の変更
    name = name.replace('<', '(back)')
    name = name + '_'
    
    count = 0
    for file in files:
        index = re.search(name, file)
        if index: count+=1
        
    name = name + str(count+1)
    
    return name

In [96]:
dictFix = readPickle('fixationTotalDict.pkl')
dictTracing = readPickle('tracingTotalDict.pkl')
folder = 'result3'

for person in dictFix.keys():
    for block in dictFix[person].keys():
        for sentence in dictFix[person][block].keys():
    
##########

            spacePlace = [] #スペースの位置を保管
    
            try:
                for space in range(len(dictTracing[person][block][sentence]['key'])):
                    if dictTracing[person][block][sentence]['key'][space] is ' ' : spacePlace.append(space)
                        
            except: continue
   
                #####

            #値の初期化
            start = 1 #wordの始まり

            #空白までの情報を取る
            for space in range(len(spacePlace)): 
                xfFix = [] #Fixaction xf
                yfFix = [] #Fixaction yf
                xfTracing = [] #Tracing xf
                yfTracing = [] #Tracing yf

                name = dictFix[person][block][sentence]['key'][start-1] #ファイル名
                for pick in range(start, spacePlace[space]):
                    name = name + dictFix[person][block][sentence]['key'][pick]
                    xfFix.append(dictFix[person][block][sentence]['xf'][pick])
                    yfFix.append(dictFix[person][block][sentence]['yf'][pick])
                    xfTracing.append(dictTracing[person][block][sentence]['xf'][pick])
                    yfTracing.append(dictTracing[person][block][sentence]['yf'][pick])

                #####

                        
                name = str(person) + '_' + str(block) + '_' + str(sentence) + '_' + name 
            
                
                #画像に描写する
                drawPicture(name, xfFix, yfFix, xfTracing, yfTracing, folder)

                #####
        
                start = spacePlace[space]+2
    break
break

KeyboardInterrupt: 